<a href="https://colab.research.google.com/github/Sarahmela93/AirBNB_THP/blob/main/Netflix_Reputation_Analysis_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Netflix Reputation Analysis 2024
## Web Scraping & Sentiment Classification

### Project Team:
- SOUISSI Youssef
- MELAIKIA Sarah
- Derradji Mourad
- BOUCHAREB Moncef Amine

### Last Update: [25/11/2024] Moncef

Scrapping Twitter et Google news Format CSV

In [ ]:
import tweepy
from textblob import TextBlob
import pandas as pd
import re
import unicodedata
from datetime import datetime
import time

# Configuration de l'authentification Twitter
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAANetxAEAAAAAT7tuge58Eod19VVvnxCKjXtVvno%3D5e8CCYViSaF3IcSDSC36jpafOMbQ0ZVKVqqGSHcxIkzRkL1tY5"

# Configuration de l'authentification
def configurer_client():
    """
    Configurer le client Twitter avec gestion des erreurs
    """
    try:
        client = tweepy.Client(bearer_token=BEARER_TOKEN)
        return client
    except Exception as e:
        print(f"Erreur de configuration du client : {e}")
        return None

def nettoyer_texte(texte):
    """
    Nettoie et formate le texte
    """
    # Convertir en chaîne si ce n'est pas déjà le cas
    texte = str(texte)

    # Supprimer les liens
    texte = re.sub(r'http\S+', '', texte)

    # Normaliser les caractères Unicode
    texte = unicodedata.normalize('NFKD', texte)

    # Supprimer les caractères non-imprimables et contrôles
    texte = ''.join(char for char in texte if unicodedata.category(char)[0] not in ['C', 'Z'])

    # Nettoyer les espaces multiples
    texte = re.sub(r'\s+', ' ', texte).strip()

    return texte

def analyser_sentiment(texte):
    """
    Analyse le sentiment d'un texte
    """
    sentiment_analysis = TextBlob(texte)
    sentiment_score = sentiment_analysis.sentiment.polarity

    if sentiment_score > 0.5:
        return "Très Positif"
    elif 0 < sentiment_score <= 0.5:
        return "Positif"
    elif sentiment_score == 0:
        return "Neutre"
    elif -0.5 < sentiment_score < 0:
        return "Négatif"
    else:
        return "Très Négatif"

def extraire_hashtags(texte):
    """
    Extrait les hashtags du texte
    """
    return ' '.join(re.findall(r"#(\w+)", texte))

def recuperer_tweets(client, query, max_tentatives=3):
    """
    Récupérer les tweets avec gestion des erreurs et des limites de requêtes
    """
    for tentative in range(max_tentatives):
        try:
            # Recherche des tweets récents avec gestion des erreurs
            tweets = client.search_recent_tweets(
                query=query,
                max_results=20,
                tweet_fields=["author_id", "created_at", "text", "public_metrics", "id"],
                expansions=["author_id"]
            )

            # Vérifier si des tweets ont été trouvés
            if tweets.data:
                return tweets

            print(f"Aucun tweet trouvé (Tentative {tentative + 1})")
            time.sleep(2 ** tentative)  # Backoff exponentiel

        except tweepy.TooManyRequests:
            print(f"Limite de requêtes atteinte (Tentative {tentative + 1})")
            time.sleep(15 * 60)  # Attendre 15 minutes

        except Exception as e:
            print(f"Erreur lors de la récupération : {e}")
            time.sleep(2 ** tentative)

    return None

def collecter_donnees_twitter(query):
    """
    Collecter et traiter les données Twitter
    """
    # Configurer le client
    client = configurer_client()
    if not client:
        return []

    # Récupérer les tweets
    tweets_response = recuperer_tweets(client, query)

    # Liste pour stocker les résultats
    results = []

    if tweets_response and tweets_response.data:
        # Créer un dictionnaire des auteurs
        users = {user.id: user.username for user in tweets_response.includes['users']} if tweets_response.includes else {}

        # Traitement des tweets
        for tweet in tweets_response.data:
            # Texte nettoyé
            texte_nettoye = nettoyer_texte(tweet.text)

            # Construction du lien du tweet
            tweet_link = f"https://twitter.com/{users.get(tweet.author_id, 'user')}/status/{tweet.id}"

            # Ajout des informations dans la liste des résultats
            results.append({
                "ID Tweet": tweet.id,
                "Auteur": users.get(tweet.author_id, "Inconnu"),
                "Date": tweet.created_at.strftime("%Y-%m-%d %H:%M:%S"),
                "Texte Original": tweet.text,
                "Texte Nettoyé": texte_nettoye,
                "Hashtags": extraire_hashtags(tweet.text),
                "Nombre de Likes": tweet.public_metrics.get("like_count", 0),
                "Nombre de Retweets": tweet.public_metrics.get("retweet_count", 0),
                "Sentiment": analyser_sentiment(tweet.text),
                "Score de Sentiment": round(TextBlob(tweet.text).sentiment.polarity, 3),
                "Lien du Tweet": tweet_link
            })

    return results

def main():
    # Requête de recherche
    query = "@Netflix Arcane -is:retweet lang:en"

    # Collecter les données
    tweet_data = collecter_donnees_twitter(query)

    # Vérifier si des données ont été collectées
    if tweet_data:
        # Créer le DataFrame
        df = pd.DataFrame(tweet_data)

        # Réorganisation des colonnes
        colonnes_ordre = [
            "ID Tweet", "Auteur", "Date",
            "Texte Original", "Texte Nettoyé", "Hashtags",
            "Nombre de Likes", "Nombre de Retweets",
            "Sentiment", "Score de Sentiment", "Lien du Tweet"
        ]

        # Sélectionner et ordonner les colonnes
        df = df[colonnes_ordre]

        # Affichage console
        print("\n--- Analyse des Tweets ---")
        print(df.to_string(index=False))

        # Statistiques de sentiment
        print("\n--- Répartition des Sentiments ---")
        print(df["Sentiment"].value_counts())

        # Nom de fichier avec date
        date_fichier = datetime.now().strftime("%Y%m%d_%H%M%S")

        # Chemins de fichiers
        fichier_excel = f'tweets_netflix_arcane_{date_fichier}.xlsx'
        fichier_csv = f'tweets_netflix_arcane_{date_fichier}.csv'

        try:
            # Export Excel
            df.to_excel(fichier_excel, index=False, engine='openpyxl')

            # Export CSV
            df.to_csv(fichier_csv, index=False)

            print("\n--- Export réussi ---")
            print(f"Fichiers créés :")
            print(f"1. {fichier_excel}")
            print(f"2. {fichier_csv}")

        except Exception as e:
            print(f"Erreur lors de l'export : {e}")

    else:
        print("Aucune donnée n'a pu être récupérée.")


if __name__ == "__main__":
    main()

Limite de requêtes atteinte (Tentative 1)
Limite de requêtes atteinte (Tentative 2)
Limite de requêtes atteinte (Tentative 3)
